In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'pandas'

In [9]:
# Load the data
df = pd.read_csv('labeled-data.csv')

# Exclude examples labeled 'N' or 'Done'
df_filtered = df[(df['label'] != 'N') & (df['label'] != 'Done')]

# Extract features and labels
X = df_filtered.iloc[:, 2:9].values 
y = df_filtered['label'].values

NameError: name 'pd' is not defined

In [63]:
# Specify classes
class_order = ['L', 'R'] 

# Init and fit labelencoder
label_encoder = LabelEncoder()
label_encoder.fit(class_order)

# Transform labels to encoded format
y_encoded = label_encoder.transform(y)

In [64]:
# train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

# Reshape for 1D cnn
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [65]:
# X_train.shape[1] is the number of features in your data
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())  # Flatten the output before the Dense layer
model.add(Dense(64, activation='relu'))  # Second layer is Dense with 64 neurons and ReLU activation
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification


In [66]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [67]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
25/25 [==============================] - 0s 5ms/step - loss: 0.6039 - accuracy: 0.6873 - val_loss: 0.5893 - val_accuracy: 0.6742
Epoch 2/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5716 - accuracy: 0.6948 - val_loss: 0.5837 - val_accuracy: 0.6742
Epoch 3/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5536 - accuracy: 0.7112 - val_loss: 0.5702 - val_accuracy: 0.6629
Epoch 4/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5459 - accuracy: 0.7251 - val_loss: 0.5770 - val_accuracy: 0.6629
Epoch 5/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5312 - accuracy: 0.7352 - val_loss: 0.5830 - val_accuracy: 0.6854
Epoch 6/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5220 - accuracy: 0.7364 - val_loss: 0.5807 - val_accuracy: 0.6854
Epoch 7/10
25/25 [==============================] - 0s 1ms/step - loss: 0.5191 - accuracy: 0.7352 - val_loss: 0.6022 - val_accuracy: 0.6854
Epoch 8/10
25/25 [==

In [68]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Test Accuracy: {accuracy}')

4/4 [==============================] - 0s 1ms/step
Test Accuracy: 0.7346938775510204


In [2]:
%pip install pylsl

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np

def preprocess(sample):
    #sample is a list of EEG values from the electrodes
    # Convert to a NumPy array
    sample_array = np.array(sample)

    # Compute the average of the sample array
    avg_sample = np.mean(sample_array, axis=0)

    # Return the average sample
    return avg_sample


In [ ]:
import pylsl
import time

# Find the stream
print("Looking for an EEG stream...")
streams = pylsl.resolve_streams(wait_time=10.0)
if len(streams) == 0:
    print("No EEG streams found")
else:
    print(f"Found {len(streams)} streams. Connecting to the first")
    inlet = pylsl.StreamInlet(streams[0])
# Create a new inlet to read from the stream as described in docs
last_time = time.time()
prediction_interval = 1  # seconds - same as examples

while True:
    current_time = time.time()
    if current_time - last_time >= prediction_interval:

        sample, timestamp = inlet.pull_sample()
        # need to  write pre process code depends on output
        processed_sample = preprocess(sample)

        # fill in correct model here for the "--": prediction = --.predict(processed_sample)
        
        # Display the prediction
        print(f"Prediction at {timestamp}: {prediction}")

        # Update the last prediction time
        last_time = current_time
